# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,kievka,50.2619,71.5514,1.51,79,93,3.42,KZ,1712020447
1,1,lagos,6.5833,3.7500,26.83,88,44,1.17,NG,1712020110
2,2,san antonio de pale,-1.4014,5.6325,28.83,78,62,3.75,GQ,1712020447
3,3,margaret river,-33.9500,115.0667,14.96,60,0,5.51,AU,1712020448
4,4,edinburgh of the seven seas,-37.0676,-12.3116,18.61,93,31,7.83,SH,1712020448


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
#https://hvplot.holoviz.org/user_guide/Geographic_Data.html
#https://stackoverflow.com/questions/70994916/hvplot-points-with-geo-true-does-not-show-the-map
# %%capture --no-display

# Configure the map plot


mappy=city_data_df.hvplot.points('Lng','Lat',s='Humidity',color='City',alpha=.65,geo=True,tiles=True,xlabel='Longitude',ylabel='Latitude',
                                hover_cols=['Country','Cloudiness','Wind Speed'])

# Display the map
mappy


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Cloudiness,Wind Speed)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
#criteria:temperature between 21 to 30,Humidity is less than 80%, windspeed is less then 4m/s, cloudiness<75

crit_df=city_data_df.loc[(city_data_df['Max Temp']>21) & (city_data_df['Max Temp']<30) & 
                         (city_data_df['Humidity']<80) & (city_data_df['Wind Speed']<4)
                        &(city_data_df['Cloudiness']<75)].dropna()


# Display sample data
crit_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
2,2,san antonio de pale,-1.4014,5.6325,28.83,78,62,3.75,GQ,1712020447
25,25,ghat,24.9647,10.1728,24.52,10,0,0.80,LY,1712020454
34,34,kalianget,-7.0500,113.9333,28.54,75,59,0.86,ID,1712020457
45,45,awjilah,29.1081,21.2869,23.65,28,0,2.96,LY,1712020462
63,63,aswan,24.0934,32.9070,24.56,35,0,3.76,EG,1712020469
114,114,waspan,14.7419,-83.9717,25.83,79,3,3.49,NI,1712020486
155,155,san pedro pochutla,15.7333,-96.4667,28.84,66,2,3.69,MX,1712020498
197,197,vallenar,-28.5708,-70.7581,22.98,53,0,0.81,CL,1712020511
226,226,east ballina,-28.8667,153.5833,27.67,65,0,3.10,AU,1712020521
242,242,cambria,35.5641,-121.0808,22.41,79,0,2.68,US,1712020528


#### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df=crit_df.loc[:,['City','Country','Lat','Lng','Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name']=''



# Display sample data
hotel_df.head(10)


,City,Country,Lat,Lng,Humidity,Hotel Name
2,san antonio de pale,GQ,-1.4014,5.6325,78,
25,ghat,LY,24.9647,10.1728,10,
34,kalianget,ID,-7.0500,113.9333,75,
45,awjilah,LY,29.1081,21.2869,28,
63,aswan,EG,24.0934,32.9070,35,
114,waspan,NI,14.7419,-83.9717,79,
155,san pedro pochutla,MX,15.7333,-96.4667,66,
197,vallenar,CL,-28.5708,-70.7581,53,
226,east ballina,AU,-28.8667,153.5833,65,
242,cambria,US,35.5641,-121.0808,79,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
categories='accommodation.hotel'
limit=1

params = {'categories':categories,
          'apiKey':geoapify_key,'limit':limit}

print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude=hotel_df.loc[index,'Lat']
    longitude=hotel_df.loc[index,'Lng']
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url,params)
    
    # Convert the API response to JSON format
    name_address = requests.get(base_url,params).json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
san antonio de pale - nearest hotel: No hotel found
ghat - nearest hotel: فندق تاسيلي
kalianget - nearest hotel: No hotel found
awjilah - nearest hotel: No hotel found
aswan - nearest hotel: Oscar Hotel
waspan - nearest hotel: Hospedaje Vanessa
san pedro pochutla - nearest hotel: Olimpia
vallenar - nearest hotel: Hotel del Marqués
east ballina - nearest hotel: Ballina Beach Resort
cambria - nearest hotel: Olallieberry Inn
tura - nearest hotel: Hotel Polo Orchid
hojai - nearest hotel: No hotel found
bardai - nearest hotel: No hotel found
chirilagua - nearest hotel: Hotel Martinez
baiti - nearest hotel: No hotel found
luang namtha - nearest hotel: Khounxaisavanh Guesthouse
san rafael del sur - nearest hotel: Barceló Montelimar
mount isa - nearest hotel: Ibis Styles
port macquarie - nearest hotel: The Observatory
la gi - nearest hotel: Lagi resort
hadithah - nearest hotel: No hotel found
barpeta - nearest hotel: Manas Hotel
san luis de la loma - nearest hotel: No hot

,City,Country,Lat,Lng,Humidity,Hotel Name
2,san antonio de pale,GQ,-1.4014,5.6325,78,No hotel found
25,ghat,LY,24.9647,10.1728,10,فندق تاسيلي
34,kalianget,ID,-7.0500,113.9333,75,No hotel found
45,awjilah,LY,29.1081,21.2869,28,No hotel found
63,aswan,EG,24.0934,32.9070,35,Oscar Hotel
114,waspan,NI,14.7419,-83.9717,79,Hospedaje Vanessa
155,san pedro pochutla,MX,15.7333,-96.4667,66,Olimpia
197,vallenar,CL,-28.5708,-70.7581,53,Hotel del Marqués
226,east ballina,AU,-28.8667,153.5833,65,Ballina Beach Resort
242,cambria,US,35.5641,-121.0808,79,Olallieberry Inn


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display
#https://stackoverflow.com/questions/59678780/show-extra-columns-when-hovering-in-a-scatter-plot-with-hvplot#:~:text=You%20can%20use%20keyword%20hover_cols%20to%20add%20additional%20columns%20to%20your%20hover.&text=Additional%20columns%20to%20add%20to,indexes%20if%20use_index%20is%20True).

# Configure the map plot
hotel_mappy=hotel_df.hvplot.points('Lng','Lat',s='Humidity',color='City',alpha=1,geo=True,tiles=True,xlabel='Longitude',ylabel='Latitude',
                                   xlim=(-180,180),ylim=(-60,60),hover_cols=['Country','Hotel Name'])


# Display the map
hotel_mappy

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)